# Value-at-Risk for Stocks: Delta-Normal Approach, EWMA

### Lecture Notes by Jakov Ivan S. Dumbrique (jdumbrique@ateneo.edu)

MA 195L.2: Introduction to Financial Mathematics II \
Second Semester, S.Y. 2020-2021 \
Ateneo de Manila University

In [1]:
import numpy as np # Numerical Computing
import pandas as pd # Data wrangling
import matplotlib.pyplot as plt # Plotting
from scipy.stats import norm # statistical analysis
from statistics import NormalDist # statistical analysis

%matplotlib inline

# EWMA for a single-asset portfolio
The one-day 99\% VaR on the portfolio is given by
		\begin{equation}
		\text{VaR}_{0.99} = |V| = NS_0\sigma_n\Phi^{-1}(0.99).
		\end{equation}
 The variance estimate $\sigma_n^2$ from the EWMA can be used as the variance of the random variable $R$, and so we assume that $R\sim N(0,\sigma_n^2)$.
 \begin{equation}
		\sigma_n^2 \approx \sum_{i=1}^{n-1}(1-\lambda)\lambda^{i-1}R_{n-i}^2,
\end{equation}
	giving us $\alpha_i = (1-\lambda)\lambda^{i-1}$, $i=1,2,\dots,n-1$.
    
$\alpha_i = \alpha_{j}= (1-\lambda)\lambda^{j}$, $j=0,1,2,\dots,n-2$

## Question:
Today is February 23, 2018. You are a portfolio risk manager who is assigned to analyze the market risk of a portfolio of 700 PLDT (TEL) shares. Assume that the one-day volatility of the stock follows an exponentially-weighted moving average model with parameter  $\lambda$=0.65
 . Determine the portfolio's one-day 99% VaR.
 


In [2]:
def get_return(df):
    return np.log(df["close"]/df["previous"])

In [3]:
def get_weights_df(df, d, ewma_par):
    """ 
    appends weights series to df
    """
    count_returns = len(df["close"])-d
    weight_lst = [(1-ewma_par)*(ewma_par**j) for j in range(count_returns)]
    df["weight"] = pd.Series(weight_lst)
    return df

In [4]:
def d_day_alpha_percent_VaR_single_stock_delta_normal(df, d, N, alpha, ewma_par=None):
    
    """Returns the d-day 100(alpha)% VaR of a single stock using Delta Normal Approach.
    
    Parameters
    ----------
    df : pandas.DataFrame
        has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row  
    d : int
        value of d for the d-day VaR
    N : int
        number of shares for the sole stock
    alpha : int
        the value to be used in calculting the 100(alpha)% VaR (e.g. 0.99, 0.95)
    ewma_par :float
        the value of the lambda parameter in an EWMA model for the volatility of the stock
        assumes the value is in the range (0,1)
    
    Returns
    -------
    float (2 decimal places)
        d-day 100(alpha)% VaR of a single stock using Delta Normal Approach
    """

    S_0 = df.loc[0, "close"]
    
    # creates a new column that shifts the prices d values up (or -d values down)
    df["previous"] = df["close"].shift(-d)
    # get the daily returns
    df["return"] = df.apply(get_return, axis=1)
    
    #Case 1: if I will use EWMA
    if ewma_par: 
        df = get_weights_df(df, d, ewma_par)
        sigma = np.sqrt(np.nansum((df["return"]**2) * df["weight"]))
    #Case 2: I'll use the normal VaR
    else:   
        # standard deviation of your stock returns
        sigma = df["return"].std()
    
    quantile = NormalDist().inv_cdf(alpha)
    var = N * S_0 * sigma * quantile
    
    return round(var, 2)
    

In [5]:
# read the data
TEL_df = pd.read_csv("../data/TEL_2018.csv") 

d_day_alpha_percent_VaR_single_stock_delta_normal(df=TEL_df, d=1, N=700, alpha=0.99, ewma_par=0.65)

41212.93